In [ ]:
from dask.distributed import LocalCluster, Client
import xarray as xr
import dask
import intake
import os
#import fsspec, os, netrc, aiohttp,dask
from satsearch import Search
import hvplot.pandas
import hvplot.xarray
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# AUTHENTICATION CONFIGURATION
from netrc import netrc
from subprocess import Popen
from getpass import getpass

urs = 'urs.earthdata.nasa.gov'    # Earthdata URL to call for authentication
prompts = ['Enter NASA Earthdata Login Username \n(or create an account at urs.earthdata.nasa.gov): ',
           'Enter NASA Earthdata Login Password: ']

# Determine if netrc file exists, and if so, if it includes NASA Earthdata Login Credentials
try:
    netrcDir = os.path.expanduser("~/.netrc")
    netrc(netrcDir).authenticators(urs)[0]
    del netrcDir

# Below, create a netrc file and prompt user for NASA Earthdata Login Username and Password
except FileNotFoundError:
    homeDir = os.path.expanduser("~")
    Popen('touch {0}.netrc | chmod og-rw {0}.netrc | echo machine {1} >> {0}.netrc'.format(homeDir + os.sep, urs), shell=True)
    Popen('echo login {} >> {}.netrc'.format(getpass(prompt=prompts[0]), homeDir + os.sep), shell=True)
    Popen('echo password {} >> {}.netrc'.format(getpass(prompt=prompts[1]), homeDir + os.sep), shell=True)
    del homeDir

# Determine OS and edit netrc file if it exists but is not set up for NASA Earthdata Login
except TypeError:
    homeDir = os.path.expanduser("~")
    Popen('echo machine {1} >> {0}.netrc'.format(homeDir + os.sep, urs), shell=True)
    Popen('echo login {} >> {}.netrc'.format(getpass(prompt=prompts[0]), homeDir + os.sep), shell=True)
    Popen('echo password {} >> {}.netrc'.format(getpass(prompt=prompts[1]), homeDir + os.sep), shell=True)
    del homeDir
del urs, prompts

In [ ]:
#Setup GDAL Env for optimum performance
env = dict(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR', 
           AWS_NO_SIGN_REQUEST='YES',
           GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
           GDAL_SWATH_SIZE='200000000',
           VSI_CURL_CACHE_SIZE='200000000',
           CPL_VSIL_CURL_ALLOWED_EXTENSIONS='TIF',
           GDAL_HTTP_UNSAFESSL='YES',
           GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
           GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'))
os.environ.update(env)

#from osgeo import gdal
#gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
#gdal.SetConfigOption('AWS_NO_SIGN_REQUEST', 'YES')
#gdal.SetConfigOption('GDAL_MAX_RAW_BLOCK_CACHE_SIZE', '200000000')
#gdal.SetConfigOption('GDAL_SWATH_SIZE', '200000000')
#gdal.SetConfigOption('VSI_CURL_CACHE_SIZE', '200000000')
#gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS', 'TIF')
#gdal.SetConfigOption('GDAL_HTTP_UNSAFESSL', 'YES')
#gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE', '~/cookies.txt')
#gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')

#gdal.SetConfigOption('VSI_CACHE', 'FALSE')
#gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN', 'YES')
#gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE', os.path.expanduser('~\\cookies.txt'))
#gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', os.path.expanduser('~\\cookies.txt'))
#gdal.SetConfigOption("GDAL_HTTP_UNSAFESSL", "YES")
#gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','.tif')

In [ ]:
dask.config.set({'distributed.dashboard.link':'http://localhost:8888/proxy/8787/status'})#'https://localhost:8787/status'})
cluster = LocalCluster(threads_per_worker=2)
client = Client(cluster)
client

In [ ]:
def get_STAC_items(url, collection, dates, bbox):
    results = Search.search(url=url,
                        collections=[collection], 
                        datetime=dates,
                        bounding_box=bbox,    
                        sortby=['-properties.datetime'])

    items = results.items()
    print(f'Found {len(items)} Items')
    return intake.open_stac_item_collection(items)

In [ ]:
url = 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD' 
collection = 'C1711924822-LPCLOUD' #HLS
bbox=[-104.79107047,   40.78311181, -104.67687336,   40.87008987]
dates = '2013-01-01/2021-12-31'

cat = get_STAC_items(url,collection,dates,','.join(map(str, bbox)))

In [ ]:
cat.to_geopandas()

In [ ]:
@dask.delayed
def lazy_open(href,chunks=dict(band=1, x=3660, y=3660)):
    return xr.open_rasterio(href, chunks=chunks)

band = 'B01'
dataArrays = dask.compute(*[lazy_open(cat[item][band].urlpath) for item in list(cat)], retries=3)
da = xr.concat(dataArrays, dim='band', join='override', combine_attrs='drop').rename(band='time')
da['time'] = [cat[item].metadata['datetime'] for item in list(cat)]
da

In [ ]:
da = da.persist()
da